In [ ]:
# <-- Import libraries, custom functions, and load configuration & datasets <-- #

import yaml
import datetime as dt
import re
import pandas as pd
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import seaborn as sns

from statsmodels.multivariate.manova import MANOVA
from scipy import stats
from scipy.stats import pearsonr, boxcox, chi2_contingency, shapiro, probplot
from scipy.stats.contingency import association

sns.set(style="whitegrid")

# <-- Imports custom preprocessing functions from 'functions.py' <-- #

# from functions import ()

# <-- Loads YAML configuration to dynamically reference CSV output files. <-- #

config = None  # <-- Initialize config
try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("Yaml configuration file not found!")

df_demo_test = pd.read_csv(config['output_data']['file3'])
df_demo_control = pd.read_csv(config['output_data']['file4'])
df_web_data = pd.read_csv(config['output_data']['file5'])


In [91]:
# ========= Config =========
STEP_ORDER = ['start','step_1','step_2','step_3','confirm']
STEP_MAP   = {s:i for i,s in enumerate(STEP_ORDER)}
KEY = ['client_id','visitor_id','visit_id']   # key to the process

def _to_utc(series: pd.Series) -> pd.Series:
    s = pd.to_datetime(series, errors='coerce')
    if getattr(s.dt, "tz", None) is None:
        return s.dt.tz_localize("UTC")
    return s.dt.tz_convert("UTC")

def _standardize_steps(df):
    w = df.copy()
    w['process_step'] = (w['process_step'].str.lower().str.strip()
                         .str.replace(' ', '_')
                         .str.replace('step1','step_1')
                         .str.replace('step2','step_2')
                         .str.replace('step3','step_3'))
    return w[w['process_step'].isin(STEP_ORDER)]


In [92]:
# ========= 1) Filtered to the group and normalization =========
def prepare_web_for_group(df_web_data, df_demo_group):
    ids = set(df_demo_group['client_id'].unique())
    w = df_web_data[df_web_data['client_id'].isin(ids)].copy()
    w = _standardize_steps(w)
    w['date_time'] = _to_utc(w['date_time'])
    # Order **by process**
    w = w.sort_values(KEY + ['date_time']).reset_index(drop=True)
    return w

In [93]:
# ========= 2) Complete sequence → step-backs =========
def compute_back_jumps(w_full: pd.DataFrame) -> pd.DataFrame:
    wf = w_full.copy()
    wf['step_idx'] = wf['process_step'].map(STEP_MAP).astype('Int64')
    wf['prev_step_idx'] = wf.groupby(KEY)['step_idx'].shift(1)
    wf['delta'] = wf['step_idx'] - wf['prev_step_idx']
    wf['is_back_jump'] = wf['delta'].lt(0)
    back = (wf.groupby(KEY)
              .agg(n_back_jumps=('is_back_jump','sum'))
              .reset_index())
    return wf, back

In [94]:
# ========= 3) Last occurrences per step and last confirm =========
def collapse_last_per_step_and_last_confirm(wf: pd.DataFrame) -> pd.DataFrame:
    # We remove intermediate confirms per process (leaving only the last one)
    is_last_confirm = (wf['process_step'].eq('confirm') &
                       ~wf.duplicated(subset=KEY+['process_step'], keep='last'))
    drop_mask = wf['process_step'].eq('confirm') & (~is_last_confirm)
    wf2 = wf.loc[~drop_mask].copy()

    # For completion times we take the **last** occurrence of each step per process
    wf2 = (wf2.sort_values(KEY + ['process_step','date_time'])
              .drop_duplicates(subset=KEY+['process_step'], keep='last'))
    return wf2

In [95]:
# ========= 4) Table by process (1 row = 1 process) =========
def summarize_processes(wf2: pd.DataFrame, back: pd.DataFrame) -> pd.DataFrame:
    # Pivot de timestamps
    pv = wf2.pivot_table(index=KEY, columns='process_step', values='date_time', aggfunc='last') \
            .reindex(columns=STEP_ORDER)
    for c in STEP_ORDER:
        if c not in pv.columns:
            pv[c] = pd.NaT
    # all as naive UTC for subtractions
    for c in pv.columns:
        pv[c] = _to_utc(pv[c]).dt.tz_convert('UTC').dt.tz_localize(None)

    # flags reached and completed
    flags = pv.notna().astype(int).add_prefix('reached_')
    flags['completed'] = flags['reached_confirm']

    # times (min)
    def mins(b,a): return (pv[b] - pv[a]) / pd.Timedelta(minutes=1)
    times = pd.DataFrame(index=pv.index)
    times['t_start_step1'] = mins('step_1','start')
    times['t_step1_step2'] = mins('step_2','step_1')
    times['t_step2_step3'] = mins('step_3','step_2')
    times['t_step3_conf']  = mins('confirm','step_3')
    times['t_total']       = mins('confirm','start')

    proc = (flags.join(times)).reset_index()
    proc = proc.merge(back, on=KEY, how='left').fillna({'n_back_jumps':0})

    # Outcome:
    # - successful: completed and no step-backs
    # - completed_with_errors: completed with step-backs
    # - fail: no completed
    proc['outcome'] = np.where(
        (proc['completed']==1) & (proc['n_back_jumps']==0), 'successful',
        np.where((proc['completed']==1) & (proc['n_back_jumps']>0), 'completed_with_errors', 'fail')
    )
    return proc

In [96]:
# ========= 5) KPIs & Drop-off =========
def kpis_from_processes(proc: pd.DataFrame) -> pd.DataFrame:
    d = proc.copy()
    denom = max(1, int(d['reached_start'].sum()))   # who started

    kpis = {
        'n_processes'          : int(len(d)),
        'started'              : int(d['reached_start'].sum()),
        'step1_rate_%'         : 100*d['reached_step_1'].sum()/denom,
        'step2_rate_%'         : 100*d['reached_step_2'].sum()/denom,
        'step3_rate_%'         : 100*d['reached_step_3'].sum()/denom,
        'completion_rate_%'    : 100*d['completed'].sum()/denom,
        'successful_%'         : 100*(d['outcome'].eq('successful').sum())/denom,
        'completed_with_errors_%': 100*(d['outcome'].eq('completed_with_errors').sum())/denom,
        'fail%'                  : 100*(d['outcome'].eq('fail').sum())/denom,
        't_total_median_min'   : float(np.nanmedian(d['t_total'])),
        't_step1_median_min'   : float(np.nanmedian(d['t_start_step1'])),
        't_step2_median_min'   : float(np.nanmedian(d['t_step1_step2'])),
        't_step3_median_min'   : float(np.nanmedian(d['t_step2_step3'])),
        't_conf_median_min'    : float(np.nanmedian(d['t_step3_conf'])),
        'avg_back_jumps'       : float(d['n_back_jumps'].mean())
    }
    return pd.DataFrame([kpis])

In [97]:
def step_dropoff_table(proc: pd.DataFrame) -> pd.DataFrame:
    d = proc.copy()
    rows = [
        ("start→step_1", int(d['reached_start'].sum()),   int(d['reached_step_1'].sum())),
        ("step_1→step_2", int(d['reached_step_1'].sum()), int(d['reached_step_2'].sum())),
        ("step_2→step_3", int(d['reached_step_2'].sum()), int(d['reached_step_3'].sum())),
        ("step_3→confirm", int(d['reached_step_3'].sum()), int(d['completed'].sum())),
    ]
    out = pd.DataFrame(rows, columns=['from_to','n_from','n_to'])
    out['conv_rate_%'] = 100 * out['n_to'] / out['n_from'].replace({0:np.nan})
    out['conv_rate_%'] = out['conv_rate_%'].fillna(0.0)
    out['dropoff_%'] = 100 - out['conv_rate_%']
    return out

In [98]:

def _starnger_things(proc_df: pd.DataFrame, key_cols=('client_id','visitor_id','visit_id')):
    '''
    Cleans up funnel inconsistencies in an already summarized process DataFrame (e.g., proc_control).
    Rules:
    - completed==1 but reached_start==0
    - Step reached with previous steps not reached.
    Returns:
    proc_clean, proc_anomalies
    '''
    df = proc_df.copy()

    # Ensures required columns (if missing, creates them with 0)
    needed = ['reached_start','reached_step_1','reached_step_2','reached_step_3','reached_confirm','completed']
    for c in needed:
        if c not in df.columns:
            df[c] = 0

    # Masks of inconsistencies
    m_step1_prev = (df['reached_step_1'].eq(1) & df['reached_start'].eq(0))
    m_step2_prev = (df['reached_step_2'].eq(1) &
                    ((df['reached_start'].eq(0)) | (df['reached_step_1'].eq(0))))

    m_step3_prev = (df['reached_step_3'].eq(1) &
                    ((df['reached_start'].eq(0)) | (df['reached_step_1'].eq(0)) | (df['reached_step_2'].eq(0))))

    m_conf_prev  = (df['reached_confirm'].eq(1) &
                    ((df['reached_start'].eq(0)) | (df['reached_step_1'].eq(0)) | (df['reached_step_2'].eq(0)) | (df['reached_step_3'].eq(0))))

    # Full mask
    m_bad = m_step1_prev | m_step2_prev | m_step3_prev | m_conf_prev #| m_completed_no_start

    # Main reason (priority by order)
    reason = np.select(
        [
            m_conf_prev,
            m_step3_prev,
            m_step2_prev,
            m_step1_prev
        ],
        [
            "reached_confirm==1 with missing previous step(s)",
            "reached_step_3==1 with missing previous step(s)",
            "reached_step_2==1 with missing previous step(s)",
            "reached_step_1==1 with reached_start==0"
        ],
        default=""
    )

    df['reason'] = reason

    # Separates anomalies and cleans
    proc_anomalies = df.loc[m_bad].copy()
    proc_clean = df.loc[~m_bad].drop(columns=['reason'])

    # # Ordena columnas (opcional): keys primero
    # if all(k in proc_clean.columns for k in key_cols):
    #     front = list(key_cols)
    #     proc_clean = proc_clean[front + [c for c in proc_clean.columns if c not in front]]
    #     if not proc_anomalies.empty:
    #         proc_anomalies = proc_anomalies[front + [c for c in proc_anomalies.columns if c not in front]]

    return proc_clean.reset_index(drop=True), proc_anomalies.reset_index(drop=True)


In [99]:
# ========= 6) RUN (CONTROL) =========
w_control = prepare_web_for_group(df_web_data, df_demo_control)               # filters and normalizes
wf, back = compute_back_jumps(w_control)                                      # complete sequence → step-backs
wf2 = collapse_last_per_step_and_last_confirm(wf)                             # collapses to last occurrence per step
proc_control = summarize_processes(wf2, back)                                 # 11 row per process + outcomee
proc_control_clean, proc_control_anomalies = _starnger_things(proc_control)
kpis_control = kpis_from_processes(proc_control_clean)                              # KPIs
dropoff_control = step_dropoff_table(proc_control_clean)

display(proc_control_clean.head(15))
display(kpis_control)
display(dropoff_control)

print("Valid processes:", len(proc_control_clean))
print("Processes driven by inconsistencies:", len(proc_control_anomalies))
# display(proc_control_anomalies.head(5))

# Count distinct control clients represented in web data
print("Distinct control clients with activity:", proc_control_clean['client_id'].nunique())

# Sanity Check:
for client_id in [4079649, 4033851, 5305116, 1028, 69255]:
# Sanity checks 
      display(f"Client {client_id} processes:", 
            proc_control_clean[proc_control_clean['client_id']==client_id][KEY+['completed','n_back_jumps','outcome','t_total']])
            

,client_id,visitor_id,visit_id,reached_start,reached_step_1,reached_step_2,reached_step_3,reached_confirm,completed,t_start_step1,t_step1_step2,t_step2_step3,t_step3_conf,t_total,n_back_jumps,outcome
0,1028,42237450_62128060588,557292053_87239438319_391157,1,1,1,1,0,0,8.966667,-0.150000,-2.216667,NaN,NaN,2,fail
1,1186,446844663_31615102958,507052512_11309370126_442139,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,fail
2,1186,446844663_31615102958,795373564_99931517312_810896,1,1,1,0,0,0,0.183333,0.183333,NaN,NaN,NaN,0,fail
3,1195,766842522_69992551638,393817425_39015278493_996341,1,1,1,1,1,1,0.550000,1.633333,0.516667,1.383333,4.083333,0,successful
4,1197,753759429_54481946928,71862471_21202285428_848395,1,1,1,1,1,1,0.066667,1.133333,0.100000,0.283333,1.583333,1,completed_with_errors
5,1368,366307863_19014662045,784065271_45379483290_309335,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,fail
6,2439,607208067_70160939111,848231744_22569944243_37711,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,fail
7,2581,770616558_80928163524,182925466_27021409208_83502,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,fail
8,3647,742136411_92170694666,492965200_82386590930_211259,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,fail
9,5354,233679354_36495397627,192604721_14077630095_819213,1,1,1,1,1,1,0.216667,0.133333,0.283333,0.866667,1.500000,0,successful


,n_processes,started,step1_rate_%,step2_rate_%,step3_rate_%,completion_rate_%,successful_%,completed_with_errors_%,fail%,t_total_median_min,t_step1_median_min,t_step2_median_min,t_step3_median_min,t_conf_median_min,avg_back_jumps
0,22830,22830,74.529128,63.342094,57.275515,48.03767,36.811213,11.226456,51.96233,4.3,0.35,0.366667,1.1,1.45,0.261279


,from_to,n_from,n_to,conv_rate_%,dropoff_%
0,start→step_1,22830,17015,74.529128,25.470872
1,step_1→step_2,17015,14461,84.989715,15.010285
2,step_2→step_3,14461,13076,90.422516,9.577484
3,step_3→confirm,13076,10967,83.871214,16.128786


Valid processes: 22830
Processes driven by inconsistencies: 530
Distinct control clients with activity: 17893


'Client 4079649 processes:'

...

,client_id,visitor_id,visit_id,completed,n_back_jumps,outcome,t_total
12020,5305116,48895020_52187827152,454423043_73789478140_859004,0,0,fail,NaN
12021,5305116,87196875_82592805389,69164930_20082199445_40666,1,0,successful,4.8


'Client 1028 processes:'

,client_id,visitor_id,visit_id,completed,n_back_jumps,outcome,t_total
0,1028,42237450_62128060588,557292053_87239438319_391157,0,2,fail,NaN


'Client 69255 processes:'

,client_id,visitor_id,visit_id,completed,n_back_jumps,outcome,t_total
160,69255,340934634_44192334475,729968009_59026484937_664178,0,0,fail,NaN


In [100]:
# ========= 6) RUN (TEST) =========
w_test = prepare_web_for_group(df_web_data, df_demo_test)               # filters and normalizes
wf_test, back = compute_back_jumps(w_test)                              # complete sequence → step-backs
wf2_test = collapse_last_per_step_and_last_confirm(wf_test)             # collapses to last occurrence per step
proc_test = summarize_processes(wf2_test, back)                         # 1 row per process + outcomee
proc_test_clean, proc_test_anomalies = _starnger_things(proc_test)
kpis_test = kpis_from_processes(proc_test_clean)                        # KPIs
dropoff_test = step_dropoff_table(proc_test_clean)

display(proc_test_clean.head(15))
display(kpis_test)
display(dropoff_test)

print("Valid processes:", len(proc_test_clean))
print("Processes driven by inconsistencies:", len(proc_test_anomalies))
# display(proc_test_anomalies.head(5))

# Count distinct test clients represented in web data
print("Distinct test clients with activity:", proc_test_clean['client_id'].nunique())

# Sanity checks 
for client_id in [8320017, 6334360, 9988021, 7230446, 1516, 1643, 1702]:
      display(f"Client {client_id} processes:", 
            proc_test_clean[proc_test_clean['client_id']==client_id][KEY+['completed','n_back_jumps','outcome','t_total']])

,client_id,visitor_id,visit_id,reached_start,reached_step_1,reached_step_2,reached_step_3,reached_confirm,completed,t_start_step1,t_step1_step2,t_step2_step3,t_step3_conf,t_total,n_back_jumps,outcome
0,555,402506806_56087378777,637149525_38041617439_716659,1,1,1,1,1,1,0.116667,0.533333,1.650000,0.333333,2.633333,0,successful
1,647,66758770_53988066587,40369564_40101682850_311847,1,1,1,1,1,1,0.116667,0.300000,3.150000,2.716667,6.283333,0,successful
2,934,810392784_45004760546,7076463_57954418406_971348,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,fail
3,1516,182314299_63168583136,255400977_38039535960_779641,1,1,1,1,1,1,0.933333,0.583333,10.116667,4.266667,15.900000,0,successful
4,1516,182314299_63168583136,481123290_95510977345_707323,1,1,1,0,0,0,0.233333,-0.133333,NaN,NaN,NaN,3,fail
5,1643,910842861_51062379179,379946188_1773022140_107963,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,fail
6,1643,910842861_51062379179,633860590_96880450633_976109,1,1,0,0,0,0,-1.800000,NaN,NaN,NaN,NaN,1,fail
7,1755,230596682_29327090182,420603142_53705621002_674558,1,1,1,1,1,1,2.016667,0.316667,0.450000,0.933333,3.716667,0,successful
8,1755,230596682_29327090182,690855460_62242406510_53318,1,1,1,1,0,0,0.100000,0.850000,0.650000,NaN,NaN,0,fail
9,1836,619440135_40983729273,275887696_51740057136_798210,1,1,0,0,0,0,0.233333,NaN,NaN,NaN,NaN,1,fail


,n_processes,started,step1_rate_%,step2_rate_%,step3_rate_%,completion_rate_%,successful_%,completed_with_errors_%,fail%,t_total_median_min,t_step1_median_min,t_step2_median_min,t_step3_median_min,t_conf_median_min,avg_back_jumps
0,25935,25935,85.949489,74.33584,66.898014,55.365336,42.344322,13.021014,44.634664,3.55,0.15,0.4,1.083333,1.033333,0.479237


,from_to,n_from,n_to,conv_rate_%,dropoff_%
0,start→step_1,25935,22291,85.949489,14.050511
1,step_1→step_2,22291,19279,86.487820,13.512180
2,step_2→step_3,19279,17350,89.994294,10.005706
3,step_3→confirm,17350,14359,82.760807,17.239193


Valid processes: 25935
Processes driven by inconsistencies: 2881
Distinct test clients with activity: 21739


'Client 8320017 processes:'

...

,client_id,visitor_id,visit_id,completed,n_back_jumps,outcome,t_total
3,1516,182314299_63168583136,255400977_38039535960_779641,1,0,successful,15.9
4,1516,182314299_63168583136,481123290_95510977345_707323,0,3,fail,NaN


'Client 1643 processes:'

,client_id,visitor_id,visit_id,completed,n_back_jumps,outcome,t_total
5,1643,910842861_51062379179,379946188_1773022140_107963,0,0,fail,NaN
6,1643,910842861_51062379179,633860590_96880450633_976109,0,1,fail,NaN


'Client 1702 processes:'

,client_id,visitor_id,visit_id,completed,n_back_jumps,outcome,t_total


In [101]:
proc_control_clean.to_csv(config['output_data']['file6'], index=False, sep=",", encoding="utf-8")
proc_test_clean.to_csv(config['output_data']['file7'], index=False, sep=",", encoding="utf-8")